# MNIST
Tanay Trivedi

In [0]:
import tensorflow as tf
import numpy as np

## Data

In [2]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

x_train = mnist.train.images # Returns np.array
y_train = np.asarray(mnist.train.labels, dtype=np.int32)
x_test = mnist.test.images # Returns np.array
y_test = np.asarray(mnist.test.labels, dtype=np.int32)

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Create validation set...

In [0]:
valSetSize = .1 
valIdx = int((1-valSetSize)*len(x_train))

x_val = x_train[valIdx:]
y_val = y_train[valIdx:]
x_train = x_train[:valIdx]
y_train = y_train[:valIdx]

## Base Configuration

In [0]:
def cnn_model_fn(features, labels, mode): 
  #base configuration courtesy of  https://www.tensorflow.org/tutorials/
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=8,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 8])
    dense = tf.layers.dense(inputs=pool2_flat, units=5, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    vs=tf.trainable_variables()
    loss += tf.add_n([tf.nn.l2_loss(v) for v in vs]) * 0.001 #reg parameter
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [5]:
config=tf.estimator.RunConfig(log_step_count_steps=1000)

mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,config=config)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa0a75647d0>, '_model_dir': '/tmp/tmpSjZU_Q', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


In [6]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000);

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpSjZU_Q/model.ckpt.
INFO:tensorflow:loss = 2.3174946, step = 0
INFO:tensorflow:global_step/sec: 187.928
INFO:tensorflow:loss = 2.113658, step = 1000 (5.327 sec)
INFO:tensorflow:global_step/sec: 192.626
INFO:tensorflow:loss = 1.8529935, step = 2000 (5.190 sec)
INFO:tensorflow:global_step/sec: 192.207
INFO:tensorflow:loss = 1.7988894, step = 3000 (5.204 sec)
INFO:tensorflow:global_step/sec: 191.27
INFO:tensorflow:loss = 1.7133453, step = 4000 (5.228 sec)
INFO

### Validation

In [8]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
val_results = mnist_classifier.evaluate(input_fn=val_input_fn)
print(val_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-27-00:34:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpSjZU_Q/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-27-00:34:12
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.90636367, global_step = 20000, loss = 0.79667395
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/tmpSjZU_Q/model.ckpt-20000
{'loss': 0.79667395, 'global_step': 20000, 'accuracy': 0.90636367}


## Increase Dense Layer Size

In [0]:
def cnn_model_fn_1(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 8])
    dense = tf.layers.dense(inputs=pool2_flat, units=10, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    vs=tf.trainable_variables()
    loss += tf.add_n([tf.nn.l2_loss(v) for v in vs]) * 0.001
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [10]:
config1=tf.estimator.RunConfig(log_step_count_steps=1000)

mnist_classifier1 = tf.estimator.Estimator(
    model_fn=cnn_model_fn_1,config=config1)

train_input_fn_1 = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier1.train(
    input_fn=train_input_fn_1,
    steps=20000);

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa060962250>, '_model_dir': '/tmp/tmpR7ZEv7', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running lo

### Validation

In [11]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
val_results = mnist_classifier1.evaluate(input_fn=val_input_fn)
print(val_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-27-00:37:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpR7ZEv7/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-27-00:37:06
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9710909, global_step = 20000, loss = 0.16363102
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/tmpR7ZEv7/model.ckpt-20000
{'loss': 0.16363102, 'global_step': 20000, 'accuracy': 0.9710909}


## Increase Dense Layer Again

In [0]:
def cnn_model_fn_2(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 8])
    dense = tf.layers.dense(inputs=pool2_flat, units=1000, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    vs=tf.trainable_variables()
    loss += tf.add_n([tf.nn.l2_loss(v) for v in vs]) * 0.001
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [13]:
config2=tf.estimator.RunConfig(log_step_count_steps=1000)

mnist_classifier2 = tf.estimator.Estimator(
    model_fn=cnn_model_fn_2,config=config2)

train_input_fn_2 = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier2.train(
    input_fn=train_input_fn_2,
    steps=20000);

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa0555bfc90>, '_model_dir': '/tmp/tmpXZ6iTY', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running lo

### Validation

In [14]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
val_results = mnist_classifier2.evaluate(input_fn=val_input_fn)
print(val_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-27-00:39:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpXZ6iTY/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-27-00:39:40
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.986, global_step = 20000, loss = 0.31166527
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/tmpXZ6iTY/model.ckpt-20000
{'loss': 0.31166527, 'global_step': 20000, 'accuracy': 0.986}


## Test

In [15]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
test_results = mnist_classifier2.evaluate(input_fn=test_input_fn)
print(test_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-27-00:39:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpXZ6iTY/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-27-00:39:45
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9847, global_step = 20000, loss = 0.304998
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/tmpXZ6iTY/model.ckpt-20000
{'loss': 0.304998, 'global_step': 20000, 'accuracy': 0.9847}
